In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("chips.txt", header=None, names=["x", "y", "type", "color"])
df['color'] = df['type'].map(lambda x: 'red' if x else 'blue')

In [ ]:
plt.scatter(df['x'], df['y'], c=df['color'])
plt.show()